### Dependencies

In [1]:
import cv2
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.python import pywrap_tensorflow

sys.path.append('..')

import video_input as vi
import model as model
import utils as utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

/home/sebas/Documentos/bootcamp/assignment1/.env/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir'] = os.path.join('..', params['data_dir'])
params['videos_folder'] = os.path.join('..', params['videos_folder'])

In [3]:
params

{'data_dir': '..\\data',
 'temp_dir': '.temp',
 'videos_folder': '..\\data/videos',
 'shuffle': True,
 'skip_frames': 6,
 'resize': [224, 224]}

### Set test image

In [3]:
img_path = os.path.join('..',params['temp_dir'], 'test_img.jpg')
img_width = 224
img_height = 224

test_img = cv2.imread(img_path)
print('Original Size',test_img.shape)

img_resize = cv2.resize(test_img,(img_width,img_height))
img_resize = img_resize[None, ...]

img_normalized = (img_resize / 255.0)
print('Resized and normalized image', img_normalized.shape)

AttributeError: 'NoneType' object has no attribute 'shape'

### Load pretrained model

In [4]:
pretrain_model = model._initialize_pretrained_model()

### Make predictions

In [5]:
video_path = os.path.join(params['videos_folder'],'videoplayback.mp4')
video_gen = vi.load_video(video_path)

In [7]:
block4_pool_features = pretrain_model.predict(next(video_gen))

In [8]:
block4_pool_features.shape

(15, 5, 5, 1536)

In [29]:
x = model._pretrained_one(block4_pool_features)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    elements = sess.run(x)

In [33]:
elements[0,...]

array([0.49111667, 0.513336  , 0.5299837 , 0.48736665, 0.5069747 ,
       0.483067  , 0.5140917 , 0.5116698 , 0.4511453 , 0.49304977,
       0.48883072, 0.4494711 , 0.4742391 , 0.49690166, 0.5055119 ,
       0.4783683 , 0.48230588, 0.5517194 , 0.49921197, 0.51972944,
       0.50787455, 0.44780937, 0.5242461 , 0.5442976 , 0.5226282 ,
       0.48349133, 0.5150849 , 0.48515472, 0.5379674 , 0.5073418 ,
       0.5044579 , 0.49657068, 0.49197307, 0.4982176 , 0.5146164 ,
       0.48660052, 0.5124666 , 0.52547204, 0.4701331 , 0.51459706,
       0.48764348, 0.52224565, 0.5057815 , 0.50141233, 0.5173896 ,
       0.5594921 , 0.5026125 , 0.4655421 , 0.5086109 , 0.51616246,
       0.50659484, 0.52577525, 0.49910122, 0.44279143, 0.4930675 ,
       0.5128241 , 0.49222168, 0.55270815, 0.5021436 , 0.49198422,
       0.53690666, 0.502867  , 0.54673606, 0.5038416 , 0.51814497,
       0.4885397 , 0.4857123 , 0.5035505 , 0.49789897, 0.51800436,
       0.45680737, 0.48156723, 0.4966005 , 0.5081052 , 0.46407

In [40]:
x = model._pretrained_two(block4_pool_features)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    elements = sess.run(x)

In [41]:
elements[0,...]

array([0.01207865, 0.00456597, 0.01241621, 0.01621264, 0.00577279,
       0.0099596 , 0.01314681, 0.01176489, 0.00640065, 0.01363329,
       0.00652337, 0.01102627, 0.00815941, 0.00752501, 0.00336708,
       0.01122649, 0.0150097 , 0.00970603, 0.00893102, 0.01236842,
       0.0156664 , 0.01234737, 0.00740969, 0.0064897 , 0.00916571,
       0.00588586, 0.00736087, 0.01000477, 0.00940295, 0.01019476,
       0.01195395, 0.01112088, 0.00688618, 0.00673314, 0.01164916,
       0.0174953 , 0.0117521 , 0.01413415, 0.00697088, 0.00894834,
       0.00598124, 0.00965149, 0.00861228, 0.00831562, 0.01461928,
       0.01193028, 0.00673633, 0.01366661, 0.01030946, 0.0077965 ,
       0.00659045, 0.00738756, 0.00951316, 0.01454755, 0.0124333 ,
       0.00671233, 0.01119893, 0.00711523, 0.01298281, 0.01026683,
       0.006384  , 0.01246846, 0.01356132, 0.00713802, 0.0081061 ,
       0.01130381, 0.00679951, 0.00841243, 0.00657527, 0.00649087,
       0.00820064, 0.01524446, 0.00912929, 0.00380662, 0.01686